In [1]:
# !pip install pysrt
# !pip install scenedetect[opencv] --upgrade
# !pip install openai
# !pip install pyperclip

In [2]:
import pysrt
import pandas as pd
import re
from scenedetect import detect, ContentDetector
from __future__ import print_function
import os
import scenedetect
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors.content_detector import ContentDetector
import cv2
from scenedetect.scene_manager import FrameTimecode
from scenedetect import VideoManager
import numpy as np
from skimage.metrics import structural_similarity as ssim
from imgurpython import ImgurClient
from tqdm import tqdm
import openai
from youtube_transcript_api import YouTubeTranscriptApi
import pyperclip

import warnings
warnings.filterwarnings("ignore")

In [3]:
video_path = "Episode 003.mp4"
transcript_path = "TVF Tripling S01E03 - 'Right, left ya seedha...'.srt"
folder_name = video_path.split('.')[0]

if not os.path.exists(folder_name):
            os.mkdir(folder_name)
else:
    pass

In [4]:
# subs = pysrt.open(transcript_path)
# print each line of dialogue
# for sub in subs:
#     print(f'Start time: {sub.start}\nEnd time: {sub.end}\nText: {sub.text}\n\n')

# load the subtitle file


def srt_to_df(filepath):
    with open(filepath, 'r') as f:
        content = f.read()
        pattern = r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\s((?:(?!\d{2}:\d{2}:\d{2},\d{3}).)*?)\n\n'
        matches = re.findall(pattern, content, re.DOTALL)
        
        df = pd.DataFrame(matches, columns=['start_time', 'end_time', 'dialogue'])
        df['dialogue'] = df['dialogue'].str.replace('\n', ' ')  # replace newlines in dialogue with spaces
        
        return pd.DataFrame(df)


def video_to_csv(filepath):
    
    scene_list = detect(filepath, ContentDetector())
    columns = ["Scene Number", "Starting Frame Time", "Starting Frame Number", "Ending Frame Time", "Ending Frame Number", "Duration of the Scene"]

    # Initialize an empty dataframe
    scene_df = pd.DataFrame(columns=columns)

    # Iterate over each scene in the list
    for i, scene in enumerate(scene_list):
        start_time = scene[0].get_timecode()
        start_frame = scene[0].get_frames()
        #fps = scene[0].get_fps()

        end_time = scene[1].get_timecode()
        end_frame = scene[1].get_frames()

        # Add a new row to the dataframe
        scene_df.loc[i] = [i+1, start_time, start_frame, end_time, end_frame, pd.to_datetime(end_time) - pd.to_datetime(start_time)]

    # scene_df.sort_values(by='Duration of the Scene', ascending = False)
    # display(scene_df)
    
    return scene_list, scene_df

def is_image_similar(imageA, imageB, threshold=0.5):
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    score = ssim(grayA, grayB)
    return score > threshold

def save_shots(s_list, filepath, folder):
    client_id = 'xyz'
    client_secret = '1234'
    client = ImgurClient(client_id, client_secret)

    scene_nos = []
    image_paths = []
    image_urls = []
    
    for i, scene in enumerate(tqdm(s_list, desc = 'Processing and Saving Shots'), start=1):
        scene_nos.append(i)
        # print('---{}---'.format(i))
        start_time = scene[0]  # start time of the scene
        end_time = scene[1]  # end time of the scene
        framerate = scene[0].get_framerate()

        duration = end_time.get_frames() - start_time.get_frames()  # duration of the scene in frames
        duration_seconds = duration / framerate  # duration of the scene in seconds

        # Determine the number of images to save based on the scene duration
        if duration_seconds > 35:
            num_images = 6
        elif duration_seconds > 25:
            num_images = 5
        elif duration_seconds > 15:
            num_images = 4
        else:
            num_images = 3

        frame_jump = duration // (num_images-1)  # number of frames to jump between saved images

        i_paths = []
        i_urls = []

        previous_images = []
        for j in range(num_images):

            video_manager = VideoManager([filepath])
            frame_time = FrameTimecode(start_time.get_frames() + j*(frame_jump-1) + 1, video_manager.get_framerate())
            # print(frame_time)
            video_manager.set_duration(start_time=frame_time, end_time=frame_time)
            # print(video_manager)

            video_manager.start()
            v_tup = video_manager.retrieve()
            #print(v_tup)
            ret_val = v_tup[0]
            frame_im = v_tup[1]
            video_manager.release()

            if type(frame_im) != type(None):
                is_similar = any(is_image_similar(frame_im, prev_im) for prev_im in previous_images)

                if not is_similar:
                    image_name = 'Scene-{}-{}.jpg'.format(i, frame_time.frame_num)
                    i_paths.append(os.path.join(folder, image_name))
                    cv2.imwrite(os.path.join(folder, image_name), frame_im)
                    # image = client.upload_from_path(os.path.join(folder, image_name), config=None, anon=True)
                    # i_urls.append(image['link'])
                    previous_images.append(frame_im)
                    # print('{} done'.format(j))

        image_paths.append(', '.join(i_paths))
        # image_urls.append(', '.join(i_urls))
        
        snapshots = pd.DataFrame({'Shot Number': scene_nos, 'Image Paths': image_paths})
        
    return snapshots


def make_final_df(transcript, scenes, snaps):

    transcript['start_time'] = pd.to_datetime(transcript['start_time'])
    transcript['end_time'] = pd.to_datetime(transcript['end_time'])
    scenes['Starting Frame Time'] = pd.to_datetime(scenes['Starting Frame Time'])
    scenes['Ending Frame Time'] = pd.to_datetime(scenes['Ending Frame Time'])

    # Assign a constant key to each dataframe and merge
    transcript['key'] = 0
    scenes['key'] = 0
    merged_df = pd.merge(scenes, transcript, on='key')

    # Filter rows
    semifinal_df = merged_df[((merged_df['Starting Frame Time'] <= merged_df['start_time']) & 
                          (merged_df['Ending Frame Time'] >= merged_df['end_time'])) |
                         ((merged_df['Starting Frame Time'] <= merged_df['end_time']) & 
                          (merged_df['Starting Frame Time'] >= merged_df['start_time']))]

    # Drop the key column
    semifinal_df = semifinal_df.drop('key', axis=1)

    final_df = pd.merge(scenes, semifinal_df, on = "Scene Number", how = 'left')
    final_df = final_df[['Scene Number', 'Starting Frame Time_x', 'Ending Frame Time_x', 'Starting Frame Number_x',
                         'Ending Frame Number_x', 'Duration of the Scene_x', 'start_time','end_time', 'dialogue']]

    final_df.columns = ['Shot Number', 'Shot Start Time', 'Shot End Time', 'Start Frame Number',
                        'End Frame Number', 'Duration of the Scene', 'Dialogue Start Time', 'Dialogue End Time', 'Dialogue']

    final_df['Shot Start Time'] = final_df['Shot Start Time'].dt.strftime('%H:%M:%S.%f')
    final_df['Shot End Time'] = final_df['Shot End Time'].dt.strftime('%H:%M:%S.%f')
    final_df['Dialogue Start Time'] = final_df['Dialogue Start Time'].dt.strftime('%H:%M:%S.%f')
    final_df['Dialogue End Time'] = final_df['Dialogue End Time'].dt.strftime('%H:%M:%S.%f')

    final_df = pd.merge(final_df, snapshots, on = 'Shot Number')
    # final_df.to_csv('{}_final_df.csv'.format(folder_name))

    return final_df

def ChatGPT_conversation(conversation, model_id = 'gpt-3.5-turbo-16k'):
    response = openai.ChatCompletion.create(
        model=model_id,
        messages=conversation
    )
    conversation.append({'role': response.choices[0].message.role, 'content': response.choices[0].message.content})
    return conversation

def brand_relevance(transcript, API_KEY, model_id = 'gpt-3.5-turbo-16k'):
    transcript.to_clipboard(index=False)
    openai.api_key = API_KEY

    conversation=[]

    prompt = """Following is the transcript of one of the episodes of a tv series. You have to identify the different contexts of the several parts of this transcript, each more than at least 3-4 dialogues long, and identify what kind of brand categories can be displayed there. Tell me the starting time and the ending time of that part, the context during that time frame, the relevant brand categories, why you think so, and a relevance score from 1-10. Give me at least 10 such instances.: {}""".format(pyperclip.paste())

    conversation.append({'role': 'user', 'content': prompt})
    conversation = ChatGPT_conversation(conversation)
    print('\n{0}: {1}\n'.format(conversation[-1]['role'].strip(), conversation[-1]['content'].strip()))

# with open('{}_transcript.tsv'.format(folder_name), 'r') as file:
#     transcript = file.read()

In [5]:
transcript_df = srt_to_df(transcript_path)
display(transcript_df)

transcript_csv_name = '{}_transcript.csv'.format(folder_name)
transcript_tsv_name = '{}_transcript.tsv'.format(folder_name)

transcript_df.to_csv(os.path.join(folder_name, transcript_csv_name), index = False)
transcript_df.to_csv(os.path.join(folder_name, transcript_tsv_name), index = False)

,start_time,end_time,dialogue
0,"00:00:50,240","00:00:51,168","Listen,"
1,"00:00:51,462","00:00:52,612",Be angry for a few days
2,"00:00:52,621","00:00:54,565",after that if Pranav says sorry go back
3,"00:00:54,954","00:00:55,827",Fighting like this
4,"00:00:55,851","00:00:57,587",I don't want to talk about it
...,...,...,...
415,"00:22:02,813","00:22:06,571",Who took the Tiago when all of us are here?
416,"00:22:07,599","00:22:08,799",RV isn't here too!
417,"00:22:09,250","00:22:10,250",Oh shit!
418,"00:22:10,591","00:22:11,591",Oh fuck!


In [6]:
scene_list, scene_df = video_to_csv(video_path)
display(scene_df)

scene_csv_name = '{}_shots.csv'.format(folder_name)
scene_df.to_csv(os.path.join(folder_name, scene_csv_name), index = False)


,Scene Number,Starting Frame Time,Starting Frame Number,Ending Frame Time,Ending Frame Number,Duration of the Scene
0,1,00:00:00.000,0,00:00:01.160,29,0 days 00:00:01.160000
1,2,00:00:01.160,29,00:00:30.243,756,0 days 00:00:29.083000
2,3,00:00:30.243,756,00:00:34.484,862,0 days 00:00:04.241000
3,4,00:00:34.484,862,00:00:36.124,903,0 days 00:00:01.640000
4,5,00:00:36.124,903,00:00:47.005,1175,0 days 00:00:10.881000
...,...,...,...,...,...,...
283,284,00:22:13.576,33336,00:22:16.216,33402,0 days 00:00:02.640000
284,285,00:22:16.216,33402,00:22:18.176,33451,0 days 00:00:01.960000
285,286,00:22:18.176,33451,00:22:24.537,33610,0 days 00:00:06.361000
286,287,00:22:24.537,33610,00:22:32.418,33807,0 days 00:00:07.881000


In [7]:
snapshots = save_shots(s_list = scene_list, filepath = video_path, folder = folder_name)

Processing and Saving Shots:   0%|                                                             | 0/288 [00:00<?, ?it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:   0%|▏                                                    | 1/288 [00:00<02:03,  2.33it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:   1%|▎                                                    | 2/288 [00:00<02:18,  2.07it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:   1%|▌                                                    | 3/288 [00:01<01:44,  2.74i

VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  11%|█████▌                                              | 31/288 [00:07<01:01,  4.21it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  11%|█████▊                                              | 32/288 [00:07<00:59,  4.28it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  11%|█████▉                                              | 33/288 [00:07<00:57,  4.47it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  12%|██████▏                                             | 34/288 [00:08<00:54,  4.67i

VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  22%|███████████▏                                        | 62/288 [00:13<00:47,  4.80it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  22%|███████████▍                                        | 63/288 [00:14<00:46,  4.84it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  22%|███████████▌                                        | 64/288 [00:14<00:45,  4.88it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  23%|███████████▋                                        | 65/288 [00:14<00:45,  4.89i

VideoManager is deprecated and will be removed.
Processing and Saving Shots:  32%|████████████████▊                                   | 93/288 [00:20<00:42,  4.56it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  33%|████████████████▉                                   | 94/288 [00:20<00:41,  4.65it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  33%|█████████████████▏                                  | 95/288 [00:20<00:42,  4.51it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  33%|█████████████████▎                                  | 96/288 [00:21<00:42,  4.49it/s]VideoManager is deprecated and will be remov

Processing and Saving Shots:  43%|█████████████████████▉                             | 124/288 [00:26<00:34,  4.74it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  43%|██████████████████████▏                            | 125/288 [00:26<00:33,  4.83it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  44%|██████████████████████▎                            | 126/288 [00:27<00:33,  4.85it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  44%|██████████████████████▍                            | 127/288 [00:27<00:33,  4.86it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be remov

VideoManager is deprecated and will be removed.
Processing and Saving Shots:  54%|███████████████████████████▍                       | 155/288 [00:33<00:32,  4.13it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  54%|███████████████████████████▌                       | 156/288 [00:33<00:30,  4.27it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  55%|███████████████████████████▊                       | 157/288 [00:34<00:30,  4.35it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  55%|███████████████████████████▉                       | 158/288 [00:34<00:29,  4.46it/s]VideoManager is deprecated and will be remov

Processing and Saving Shots:  64%|████████████████████████████████▌                  | 184/288 [00:40<00:26,  3.89it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  64%|████████████████████████████████▊                  | 185/288 [00:41<00:27,  3.72it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  65%|████████████████████████████████▉                  | 186/288 [00:41<00:25,  3.93it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  65%|█████████████████████████████████                  | 187/288 [00:41<00:25,  4.00it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be remov

Processing and Saving Shots:  75%|██████████████████████████████████████             | 215/288 [00:48<00:17,  4.06it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  75%|██████████████████████████████████████▎            | 216/288 [00:49<00:19,  3.75it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  75%|██████████████████████████████████████▍            | 217/288 [00:49<00:18,  3.89it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  76%|██████████████████████████████████████▌            | 218/288 [00:49<00:17,  4.05it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be remov

Processing and Saving Shots:  85%|███████████████████████████████████████████▌       | 246/288 [00:56<00:11,  3.54it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  86%|███████████████████████████████████████████▋       | 247/288 [00:57<00:10,  3.73it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  86%|███████████████████████████████████████████▉       | 248/288 [00:57<00:11,  3.61it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  86%|████████████████████████████████████████████       | 249/288 [00:57<00:10,  3.57it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be remov

Processing and Saving Shots:  96%|█████████████████████████████████████████████████  | 277/288 [01:05<00:03,  3.43it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  97%|█████████████████████████████████████████████████▏ | 278/288 [01:05<00:02,  3.67it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  97%|█████████████████████████████████████████████████▍ | 279/288 [01:05<00:02,  3.70it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be removed.
Processing and Saving Shots:  97%|█████████████████████████████████████████████████▌ | 280/288 [01:06<00:02,  3.78it/s]VideoManager is deprecated and will be removed.
VideoManager is deprecated and will be remov

In [8]:
final_df = make_final_df(transcript_df, scene_df, snapshots)
display(final_df)

final_csv_name = '{}_final.csv'.format(folder_name)
final_df.to_csv(os.path.join(folder_name, final_csv_name), index = False)

,Shot Number,Shot Start Time,Shot End Time,Start Frame Number,End Frame Number,Duration of the Scene,Dialogue Start Time,Dialogue End Time,Dialogue,Image Paths
0,1,00:00:00.000000,00:00:01.160000,0,29,0 days 00:00:01.160000,NaN,NaN,NaN,Episode 003\Scene-1-1.jpg
1,2,00:00:01.160000,00:00:30.243000,29,756,0 days 00:00:29.083000,NaN,NaN,NaN,"Episode 003\Scene-2-30.jpg, Episode 003\Scene-..."
2,3,00:00:30.243000,00:00:34.484000,756,862,0 days 00:00:04.241000,NaN,NaN,NaN,Episode 003\Scene-3-757.jpg
3,4,00:00:34.484000,00:00:36.124000,862,903,0 days 00:00:01.640000,NaN,NaN,NaN,Episode 003\Scene-4-863.jpg
4,5,00:00:36.124000,00:00:47.005000,903,1175,0 days 00:00:10.881000,NaN,NaN,NaN,"Episode 003\Scene-5-904.jpg, Episode 003\Scene..."
...,...,...,...,...,...,...,...,...,...,...
503,284,00:22:13.576000,00:22:16.216000,33336,33402,0 days 00:00:02.640000,NaN,NaN,NaN,Episode 003\Scene-284-33337.jpg
504,285,00:22:16.216000,00:22:18.176000,33402,33451,0 days 00:00:01.960000,00:22:14.614000,00:22:16.351000,"Baba, I'm so sorry",Episode 003\Scene-285-33403.jpg
505,286,00:22:18.176000,00:22:24.537000,33451,33610,0 days 00:00:06.361000,NaN,NaN,NaN,"Episode 003\Scene-286-33452.jpg, Episode 003\S..."
506,287,00:22:24.537000,00:22:32.418000,33610,33807,0 days 00:00:07.881000,NaN,NaN,NaN,Episode 003\Scene-287-33611.jpg


In [8]:
brand_relevance(transcript_df)


assistant: Part 1: 00:00:50,240 to 00:01:01,134
Context: A conversation between two characters about being angry and the possibility of leaving a marriage.
Brand categories: Counseling services, marriage counseling, relationship advice.
The relevance score: 7

Part 2: 00:01:12,610 to 00:01:26,493
Context: An argument between the characters, with one trying to calm the other down.
Brand categories: Counseling services, self-help books, meditation apps.
The relevance score: 6

Part 3: 00:01:27,662 to 00:01:34,376
Context: The characters arguing about the seriousness of the situation.
Brand categories: Car rental service, travel apps, navigation systems.
The relevance score: 5

Part 4: 00:03:03,079 to 00:03:13,389
Context: A conversation about the consequences of breaking off a marriage.
Brand categories: Legal services, divorce attorneys, marriage counseling.
The relevance score: 7

Part 5: 00:04:22,220 to 00:04:34,151
Context: The characters discussing their fears while lost in the des

In [7]:
transcript_df

,start_time,end_time,dialogue
0,"00:00:50,240","00:00:51,168","Listen,"
1,"00:00:51,462","00:00:52,612",Be angry for a few days
2,"00:00:52,621","00:00:54,565",after that if Pranav says sorry go back
3,"00:00:54,954","00:00:55,827",Fighting like this
4,"00:00:55,851","00:00:57,587",I don't want to talk about it
...,...,...,...
415,"00:22:02,813","00:22:06,571",Who took the Tiago when all of us are here?
416,"00:22:07,599","00:22:08,799",RV isn't here too!
417,"00:22:09,250","00:22:10,250",Oh shit!
418,"00:22:10,591","00:22:11,591",Oh fuck!
